In [4]:
import dlib
import cv2
import numpy as np


In [5]:
model_path="/home/jal/.deepface/weights/vgg_face_weights.h5"
# Load the pre-trained face detection model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../raw_data/shape_predictor_81_face_landmarks.dat')


In [6]:
import os

In [ ]:
os.getcwd()


In [46]:
folder_input_path = "../raw_data/output_imdb_top100/"
folder_output_path = "../raw_data/image_trans/"

file_list = [os.path.join(folder_input_path, file) for file in os.listdir(folder_input_path) if os.path.isfile(os.path.join(folder_input_path, file))]
print(len(file_list))

207


In [51]:
images = [f.replace('.jpg', '') for f in file_list]
for image_path in images:
    image = cv2.imread(image_path+'.jpg')
    # Convert the image to grayscale for face detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect faces in the image
    faces = detector(gray)
    # We take only the first detected face, if exist
    if len(faces) == 0:
         # Save the final preprocessed image
        save_path = image_path.replace(folder_input_path,folder_output_path)+".jpg"
        # black and white output:
        cv2.imwrite(save_path, cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        print("Image without detected face", image_path)
        continue
    face = faces[0]
    
    # We extract the landmarks   
    landmarks = predictor(image, face)
    landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

    # Calculate the center of the eyes
    left_eye = np.mean(landmarks[36:42], axis=0)
    right_eye = np.mean(landmarks[42:48], axis=0)
    nose = landmarks[30]

    # Calculate the angle between the eyes and the horizontal line
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))

    # Rotate the image around the center of the face rectangle
    (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
    center = (x + w // 2, y + h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    aligned_face = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC)
    if ((aligned_face.shape[0] == 0) or (aligned_face.shape[1] == 0) or (aligned_face.shape[2] == 0)):
        continue
    
    # Re apply the face crop on the aligned image
    gray = cv2.cvtColor(aligned_face, cv2.COLOR_BGR2GRAY)
    faces_aligned = detector(gray)
    if not faces_aligned:
        continue
    reduce = 0.7
    increase = 1/0.7
    
    x, y, w, h = int(faces_aligned[0].left()*reduce), int(faces_aligned[0].top()*reduce), int(faces_aligned[0].width()*increase), int(faces_aligned[0].height()*increase)
    processed_face = aligned_face[y:y+h, x:x+w]
    if ((processed_face.shape[0] == 0) or (processed_face.shape[1] == 0) or (processed_face.shape[2] == 0)):
        continue
    
    # Save the final preprocessed image
    save_path = image_path.replace(folder_input_path,folder_output_path)+".jpg"
    # black and white output:
    cv2.imwrite(save_path, cv2.cvtColor(processed_face, cv2.COLOR_BGR2GRAY)) #cv2.cvtColor(processed_face, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite(save_path, face_image)
    # print(f"Face image saved: {save_path}")

Image without detected face ../raw_data/output_imdb_top100/marilyn_monroe
Image without detected face ../raw_data/output_imdb_top100/george_c._scott
Image without detected face ../raw_data/output_imdb_top100/kim_basinger
Image without detected face ../raw_data/output_imdb_top100/kim_novak
Image without detected face ../raw_data/output_imdb_top100/charles_laughton
Image without detected face ../raw_data/output_imdb_top100/virginia_cherrill


## To reduce the image size of all images 

In [25]:
from PIL import Image

In [29]:
folder_path = "../raw_data/output_imdb_top100/"

file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

images = [f.replace('.jpg', '') for f in file_list]
for image_path in images:
    img = cv2.imread(image_path+'.jpg')
    width = int(img.shape[1])
    height = int(img.shape[0])
    target_width = 600
    if width > target_width:
        dim = (target_width, int(height*(target_width/width)))
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        resized = cv2.cvtColor(resized, cv2.COLOR_RGB2BGR)
        Image.fromarray(resized).save(image_path+'.jpg')


267
270
339
427
2165
1284
640
680
3186
288
317
291
4256
1080
1363
1500
1268
2000
300
600
4146
274
295
324
266
1417
1275
293
1295
309
2400
399
1363
448
1368
1311
281
306
1200
281
600
1665
214
288
1360
1641
600
2667
1371
447
271
280
2271
2358
289
274
1361
2924
3295
2400
1023
1000
291
785
1340
1010
515
2000
274
297
278
1556
289
1403
360
295
365
1665
1170
595
3795
389
596
3149
272
1551
1458
408
1976
2400
269
287
1536
1363
276
360
266
1800
2840
295
2400
312
1545
1638
288
1689
928
277
600
602
304
337
304
5184
1357
1411
2417
1361
600
1435
282
371
1361
283
1370
1363
300
267
280
421
1535
1437
1363
1326
1404
1687
1497
1365
1325
298
1419
427
1038
480
463
279
1464
1362
1365
1638
355
1360
287
347
1483
339
369
3617
800
267
1256
600
270
343
3028
266
640
338
303
316
1567
319
1671
306
313
1517
1779
1365
1389
642
1413
1671
834
283
1814
1363
2048
1370
281
1363
2048
1358
1364
1334
1426
450
304
1363
448
1307
1272
1363
286
346
645
293
274
303
355


In [33]:
folder_path = "../raw_data/output_imdb_top100/"

file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

file_list = ["../raw_data/output_imdb_top100/samara_weaving.jpg"]

images = [f.replace('.jpg', '') for f in file_list]
for image_path in images:
    img = cv2.imread(image_path+'.jpg')
    width = int(img.shape[1])
    height = int(img.shape[0])
    target_width = 600
    if width > target_width:
        dim = (target_width, int(height*(target_width/width)))
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        resized = cv2.cvtColor(resized, cv2.COLOR_RGB2BGR)
        Image.fromarray(resized).save(image_path+'.jpg')